# PDF table exptraction  

In [ ]:
import os
import sys
import tabula
import PyPDF2
import os
import pandas as pd
import numpy as np

In [ ]:
path = os.getcwd()
path = path+'/../PDFs/'

### Reading pdf files using tabula and pypdf

In [ ]:

# reading PDF file to extract tables from
file = path+"./B.pdf"
tables = tabula.read_pdf(file, multiple_tables=True, pages='16')
pdf_file = PyPDF2.PdfFileReader(open(file,'rb'))

In [ ]:
# get number of pages in pdf file 
totalPages = pdf_file.numPages
print(totalPages)


In [ ]:
 pdf_file.getDocumentInfo()

### First attempt to find and correct 90 deg rotated pages 

In [ ]:
for i in range(1,totalPages):
    #print (pdf_file.getPage(i).get('/Rotate'))
    #check if file is rotated at any angle, if yes, print the information in the file
    if (pdf_file.getPage(i).get('/Rotate') != None and (pdf_file.getPage(i).get('/Rotate')) != 0):
        print("Page: "+str(i)+" "+"Angle: "+str(pdf_file.getPage(i).get('/Rotate')))
    
    
        

In [ ]:
pdf_writer = PyPDF2.PdfFileWriter()

pdf_page = pdf_file.getPage(16)
pdf_page.rotateClockwise(90)  # rotateClockwise()
pdf_writer.addPage(pdf_page)

with open('P_rotated.pdf', 'wb') as pdf_file_rotated:
    pdf_writer.write(pdf_file_rotated)

In [ ]:
tables_rot = tabula.read_pdf('P_rotated.pdf', multiple_tables=True)

In [ ]:
pdf_file.getPage(20).get('/Rotate')

tables_rot

## cleaning tables 
1. drop based on shape - smaler than 2 in row or column will be droped
2. drop if number of NAN elements are higher than others
3. drop if all columns are objects

In [ ]:
table_clean = []
i=0
j = len(tables)
while i < j:
    #print(i)
    
    if (tables[i-1].shape[1]<2) or (tables[i-1].shape[0]<2):
        #del tables[i-1]
        #j-=1
        i+=1
        continue
    if tables[i-1].isna().sum().sum() > tables[i-1].count().sum():
        
        i+=1
        continue
    
  
    df = pd.DataFrame(tables[(i-1)])
    
    for column in df.columns:
        if (df[column].dtypes != 'object') and (df[column].sum() > df[column].isna().sum()):
            drop = False
            break
        else:
            drop = True
    #print("drop: ", drop)
    if drop:
        #del tables[i-1]
        #j-=1
        i+=1
        continue
    
    #drop columns if all values are NAN
    
    df = df.dropna(axis=1, how='all')
    
    #add to table clean
    table_clean.append(df)
    i+=1
#print(i)


In [ ]:
len(table_clean)

# Fixing Unnamed columns 
if half of the coulumn names are unnamed, use next row for names if the next row is str

In [ ]:
for num in range(len(table_clean)):
    rows_with_nan=[]
    i = 1
    j = table_clean[num].columns.size
    for column in table_clean[num].columns:
        if ("Unnamed"  in column):
            i+=1
    if i>j/2:
        table_clean[num] = table_clean[num].rename(columns=table_clean[num].iloc[0,0:])
        table_clean[num] = table_clean[num].drop([0])
    #re-indexing
    table_clean[num].index = range(len(table_clean[num]))   
    
    # filter rows with nan and add the row index to next row
    rows_with_nan = [index for index, row in table_clean[num].iterrows() if row.isnull().sum() > 2]
    
    if rows_with_nan: 
        for k in rows_with_nan:
            table_clean[num].iat[k+1,0] =  str(table_clean[num].iloc[k,0]) + "-" +  str(table_clean[num].iloc[k+1,0])
        table_clean[num] = table_clean[num].drop(rows_with_nan)
        
    table_clean[num].columns.str.upper()
    
    # if left and bottom of a column is nan move it to left and drop the column
    
    if table_clean[num].columns[0] is np.nan and table_clean[num].columns[1] is not np.nan and table_clean[num].iloc[2,1] is np.nan:
        table_clean[num] = table_clean[num].rename(columns={table_clean[num].columns[0]:str(table_clean[num].columns[1]).upper()})
        table_clean[num] = table_clean[num].drop(columns = [str(table_clean[num].columns[1])])
    
            

In [ ]:
table_clean[0]


In [ ]:
table_clean[0].columns[8].isnumeric()

In [ ]:
float(table_clean[0].columns[8])

In [ ]:
x = [column for column in table_clean[0].columns if column.isnumeric()]
len(x)

In [ ]:
rows_with_nan = [index for index, row in table_clean[1].iterrows() if row.isnull().sum() > 2]

In [ ]:
for i in rows_with_nan:
    
    table_clean[0].iat[i+1,0] =  str(table_clean[0].iloc[i,0]) + "-" +  str(table_clean[0].iloc[i+1,0])
    
#table_clean[0] = table_clean[0].drop(rows_with_nan)

In [ ]:
table_clean[0] = table_clean[0].drop(rows_with_nan)

In [ ]:
table_clean[0].columns.str.upper()

In [ ]:
rows_with_nan = []
if not rows_with_nan:
    print("X")

# 